In [ ]:
1️⃣ ReAct Agent (Reason + Act) ⭐ Most Important

The LLM:

Thinks step-by-step
Chooses tools
Uses observations
Continues until done

Best for

RAG systems
Search + tools
AI assistants
Healthcare / Finance bots

 #---------------------
🧠 ReAct Agent = Think → Act → Observe → Answer
What this example does

User asks:

“What is 10 plus 25?”

LLM will:

Think: “I need to calculate”
Act: Call add_numbers
Observe: Get 35
Answer: “35”

In [6]:
"""
REACT AGENT (LangChain v1 – Compatible Version)

This agent:
- Thinks
- Chooses a tool
- Uses observation
- Produces an answer
"""

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Load API key
load_dotenv(".env")

# LLM (Brain)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ----------- TOOL -----------
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

# ----------- STEP 1: LLM THINKS -----------
# LLM sees the question and decides to call a tool
response = llm.bind_tools([add_numbers]).invoke(
    "What is 10 plus 25?"
)

# ----------- STEP 2: TOOL IS CALLED -----------
tool_call = response.tool_calls[0]          # LLM asked to call add_numbers
tool_result = add_numbers.invoke(tool_call["args"])

# ----------- STEP 3: LLM USES OBSERVATION -----------
final = llm.invoke(
    f"The result of the calculation is {tool_result}. "
    "Give the final answer to the user."
)

print("Final Answer:", final.content)


Final Answer: The final answer is 35.


In [ ]:
2️⃣ Tool-Calling Agent (OpenAI Functions Agent) ⭐ Enterprise Standard

Uses structured JSON tool calls.

Why companies use it

Safer
Deterministic
No prompt-hacking
Works with APIs

Best for

Backend automation
Production systems
Healthcare & banking

In [7]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Load API key
load_dotenv(".env")

# LLM (Brain) – supports structured tool calling
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ---------------- TOOL ----------------
@tool
def get_square(n: int) -> int:
    """Return the square of a number"""
    return n * n

# ---------------- STEP 1: LLM DECIDES ----------------
# LLM receives the question and decides whether to call a tool
response = llm.bind_tools([get_square]).invoke(
    "What is the square of 7?"
)

# ---------------- STEP 2: TOOL IS CALLED ----------------
tool_call = response.tool_calls[0]         # Structured JSON tool call
tool_result = get_square.invoke(tool_call["args"])

# ---------------- STEP 3: LLM USES TOOL RESULT ----------------
final = llm.invoke(
    f"The square is {tool_result}. "
    "Give the final answer to the user."
)

print("Final Answer:", final.content)


Final Answer: The final answer is 7, since the square root of 49 is 7.


In [ ]:
3️⃣ Conversational Agent

An agent with memory.

Best for

Chatbots
Support bots
Personal assistants

In [8]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Load API key
load_dotenv(".env")

# LLM (Brain)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ---------------- TOOL ----------------
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

# ---------------- SIMPLE MEMORY ----------------
chat_history = []

def chat(user_input):
    # Add user message to memory
    chat_history.append(f"User: {user_input}")

    # Build full conversation
    full_prompt = "\n".join(chat_history)

    # Let LLM decide whether to call tool
    response = llm.bind_tools([add_numbers]).invoke(full_prompt)

    # If LLM wants to call a tool
    if response.tool_calls:
        tool_call = response.tool_calls[0]
        tool_result = add_numbers.invoke(tool_call["args"])

        # LLM uses tool result
        final = llm.invoke(
            f"{full_prompt}\nTool result: {tool_result}\nGive a helpful reply."
        )
        answer = final.content
    else:
        answer = response.content

    # Save assistant reply
    chat_history.append(f"Assistant: {answer}")

    return answer


# ---------------- CHAT ----------------
print(chat("Hi"))
print(chat("What is 5 plus 6?"))
print(chat("Add 10 more to that"))


Hello! How can I assist you today?
The sum of 5 plus 6 is 11. If you have any more math questions or anything else you'd like to know, feel free to ask!
The result of adding 10 to 11 is 21. If you have any more calculations or questions, just let me know!


In [ ]:
4️⃣ RAG Agent (Agent + Retriever)

Not a separate class — it is:

Agent + Vector DB Tool
The agent decides:
When to search
When to answer

Best for
Document QA
Policy bots
Knowledge bases

In [14]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Load API key
load_dotenv(".env")

# LLM (Brain)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ---------------- DOCUMENT STORE ----------------
documents = {
    "langchain": "LangChain is a framework for building applications using LLMs.",
    "langgraph": "LangGraph is used to build stateful multi-agent workflows.",
    "rag": "RAG stands for Retrieval Augmented Generation."
}

# ---------------- RETRIEVER TOOL ----------------
@tool
def search_docs(query: str) -> str:
    """
    Search internal documents and return relevant text.
    """
    for key, text in documents.items():
        if key.lower() in query.lower():
            return text
    return "No relevant document found."

# ---------------- SIMPLE MEMORY ----------------
chat_history = []

def chat(user_input):
    # Save user message
    chat_history.append(f"User: {user_input}")

    # Build conversation context
    full_prompt = "\n".join(chat_history)

    # Ask LLM – it decides whether to call search_docs
    response = llm.bind_tools([search_docs]).invoke(full_prompt)

    # If LLM wants to search documents
    if response.tool_calls:
        tool_call = response.tool_calls[0]
        retrieved_text = search_docs.invoke(tool_call["args"])

        # Give retrieved knowledge back to LLM
        final = llm.invoke(
            f"{full_prompt}\nRetrieved info: {retrieved_text}\nAnswer clearly."
        )
        answer = final.content
    else:
        answer = response.content

    # Save assistant reply
    chat_history.append(f"Assistant: {answer}")

    return answer


# ---------------- CHAT ----------------

print("What is LangChain?\n",chat("What is LangChain?"))
print("What is RAG?\n",chat("What is RAG?"))
print("What is your name\n",chat("What is your name?"))


What is LangChain?
 LangChain is a framework designed for building applications that utilize large language models (LLMs). It provides tools and components to help developers create, manage, and deploy applications that leverage the capabilities of these models.
What is RAG?
 RAG, or Retrieval Augmented Generation, is a technique that combines retrieval-based methods with generative models. It enhances the capabilities of language models by allowing them to access and incorporate external information from a knowledge base or database during the generation process. This approach helps improve the accuracy and relevance of the generated content by grounding it in real-world data.
What is your name
 I am an AI language model and don't have a personal name. You can simply refer to me as "Assistant." How can I help you today?


In [ ]:
The model is forced to answer only from documents.
If it’s not in the docs → it must say “I don’t know.”

In [19]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(".env")

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ---------------- DOCUMENT STORE ----------------
documents = {
    "langchain": "LangChain is a framework for building applications using LLMs.",
    "langgraph": "LangGraph is used to build stateful multi-agent workflows.",
    "rag": "RAG stands for Retrieval Augmented Generation."
}

# ---------------- SIMPLE RETRIEVER ----------------
def retrieve(query):
    for k, v in documents.items():
        if k in query.lower():
            return v
    return None

# ---------------- STRICT RAG ----------------
def ask(question):
    doc = retrieve(question)

    # If no document found → do NOT let LLM guess
    if not doc:
        return "I don’t know. This information is not in the documents."

    # Force LLM to use only retrieved text
    prompt = f"""
Answer ONLY using the following document.

Document:
{doc}

Question:
{question}

If the answer is not in the document, say "I don't know."
"""

    response = llm.invoke(prompt)
    return response.content


# ---------------- TEST ----------------
print("LangChain:", ask("What is LangChain?"))
print("RAG:", ask("What is RAG?"))
print("Name:", ask("What is your name?"))


LangChain: LangChain is a framework for building applications using LLMs.
RAG: RAG stands for Retrieval Augmented Generation.
Name: I don’t know. This information is not in the documents.


In [ ]:
5️⃣ Self-Ask Agent

Breaks a big question into smaller ones and searches step-by-step.

Mostly replaced by ReAct now.

Main Question
↓
Split into sub-questions
↓
Search each
↓
Combine answers


In [12]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Load API key
load_dotenv(".env")

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ---------------- DOCUMENT STORE ----------------
docs = {
    "founder": "LangChain was founded by Harrison Chase.",
    "langchain": "LangChain is a framework for building applications using LLMs."
}

# ---------------- SEARCH TOOL ----------------
@tool
def search_docs(query: str) -> str:
    """Search internal knowledge"""
    for key, value in docs.items():
        if key.lower() in query.lower():
            return value
    return "No information found."

# ---------------- SELF-ASK AGENT ----------------
def self_ask(question):
    # Step 1: Ask LLM to split into sub-questions
    breakdown = llm.invoke(
        f"Break this question into two simple questions: {question}"
    ).content

    print("\nSub-questions:")
    print(breakdown)

    # Step 2: Ask LLM to pick search terms
    sub_questions = [
        "founder",
        "langchain"
    ]

    answers = []

    # Step 3: Search for each sub-question
    for q in sub_questions:
        ans = search_docs.invoke({"query": q})
        answers.append(ans)

    # Step 4: Let LLM combine answers
    final = llm.invoke(
        f"Question: {question}\n"
        f"Answers: {answers}\n"
        "Combine these into a single clear answer."
    )

    return final.content


# ---------------- RUN ----------------
print(self_ask("Who founded LangChain and what is it?"))



Sub-questions:
1. Who founded LangChain?  
2. What is LangChain?
LangChain was founded by Harrison Chase and is a framework for building applications using large language models (LLMs).


In [ ]:
6️⃣ Plan-and-Execute Agent

Two agents:

Planner creates a plan
Executor runs steps

Best for

Long workflows
Business automation
Multi-step tasks

This shows:

One LLM plans
Another LLM executes step by step
User question
↓
Planner → creates plan
↓
Executor → runs steps
↓
Tools → get data
↓
Executor → final answer


In [13]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Load API key
load_dotenv(".env")

# Planner LLM
planner_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Executor LLM
executor_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ---------------- DOCUMENT STORE ----------------
docs = {
    "langchain": "LangChain is a framework for building applications using LLMs.",
    "langgraph": "LangGraph is a framework for building stateful multi-agent workflows."
}

# ---------------- SEARCH TOOL ----------------
@tool
def search_docs(query: str) -> str:
    """Search internal knowledge"""
    for k, v in docs.items():
        if k.lower() in query.lower():
            return v
    return "Not found."

# ---------------- PLANNER ----------------
def planner(question):
    plan = planner_llm.invoke(
        f"Create a step-by-step plan to answer: {question}"
    ).content
    return plan

# ---------------- EXECUTOR ----------------
def executor(question, plan):
    print("\nPlan:\n", plan)

    steps = ["langchain", "langgraph"]
    results = []

    for step in steps:
        result = search_docs.invoke({"query": step})
        results.append(result)

    final = executor_llm.invoke(
        f"Question: {question}\n"
        f"Results: {results}\n"
        "Combine these into a clear answer."
    )
    return final.content


# ---------------- RUN ----------------
question = "What is LangChain and what is LangGraph?"

plan = planner(question)
answer = executor(question, plan)

print("\nFinal Answer:\n", answer)



Plan:
 To effectively answer the question "What is LangChain and what is LangGraph?", you can follow this step-by-step plan:

### Step 1: Define the Scope
- **Objective**: Clearly outline what you want to achieve with your answer. You want to provide a comprehensive understanding of both LangChain and LangGraph.
- **Target Audience**: Consider who will be reading your answer (e.g., developers, researchers, general public) to tailor the complexity of your explanation.

### Step 2: Research LangChain
- **Overview**: Start by gathering basic information about LangChain. Look for its purpose, features, and applications.
- **Key Features**: Identify the main functionalities of LangChain, such as its capabilities in natural language processing, integration with other tools, and any unique selling points.
- **Use Cases**: Find examples of how LangChain is used in real-world applications or projects.
- **Official Documentation**: Check the official website or GitHub repository for the most ac

In [ ]:
7️⃣ Multi-Tool Agent

One agent using many tools:

DB
APIs
Python
Search
Vector DB

This is how real enterprise agents are built.

In [15]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Load API key
load_dotenv(".env")

# LLM (Brain)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ---------------- KNOWLEDGE BASE ----------------
docs = {
    "langchain": "LangChain is a framework for building applications using LLMs."
}

# ---------------- TOOL 1: SEARCH ----------------
@tool
def search_docs(query: str) -> str:
    """Search internal documents"""
    for k, v in docs.items():
        if k in query.lower():
            return v
    return "Not found."

# ---------------- TOOL 2: MATH ----------------
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

tools = [search_docs, add_numbers]

# ---------------- MULTI-TOOL AGENT ----------------
def ask(question):
    # LLM decides which tool to call
    response = llm.bind_tools(tools).invoke(question)

    # If tool is needed
    if response.tool_calls:
        tool_call = response.tool_calls[0]
        tool_name = tool_call["name"]
        args = tool_call["args"]

        # Run correct tool
        tool_map = {
            "search_docs": search_docs,
            "add_numbers": add_numbers
        }
        tool_result = tool_map[tool_name].invoke(args)

        # Send tool result back to LLM
        final = llm.invoke(
            f"Question: {question}\nTool result: {tool_result}\nAnswer clearly."
        )
        return final.content

    return response.content


# ---------------- RUN ----------------
print(ask("What is LangChain?"))
print(ask("What is 5 plus 7?"))


LangChain is a framework designed for developing applications that utilize large language models (LLMs).
5 plus 7 equals 12.


In [ ]:
Human-in-the-Loop Agent
Scenario:

AI calculates a value
Human must approve before final answer

In [17]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Load API key
load_dotenv(".env")

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ----------- TOOL -----------
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

# ----------- HUMAN APPROVAL TOOL -----------
def human_approval(result):
    print("\nAI suggests:", result)
    decision = input("Approve? (yes/no): ")
    return decision.lower() == "yes"

# ----------- HUMAN-IN-THE-LOOP AGENT -----------
def ask(question):
    # Step 1: LLM decides to use tool
    response = llm.bind_tools([add_numbers]).invoke(question)

    # Step 2: Run tool
    tool_call = response.tool_calls[0]
    tool_result = add_numbers.invoke(tool_call["args"])

    # Step 3: Human review
    if not human_approval(tool_result):
        return "Human rejected the AI result."

    # Step 4: Final answer after approval
    final = llm.invoke(
        f"The approved result is {tool_result}. Answer the user."
    )
    return final.content


# ---------------- RUN ----------------
print(ask("What is 40 plus 2?"))



AI suggests: 42


Approve? (yes/no):  no


Human rejected the AI result.


In [18]:
print(ask("What is 40 plus 2?"))


AI suggests: 42


Approve? (yes/no):  yes


The approved result is 42. If you have any further questions or need additional information, feel free to ask!
